# 허깅페이스 임베딩(HuggingFace Embeddings)

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH08-Embeddings")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH08-Embeddings


In [3]:
import os
import warnings

# 경고 무시
warnings.filterwarnings("ignore")

# ./cache/ 경로에 다운로드 받도록 설정
os.environ["HF_HOME"] = "./cache/"

## 샘플 데이터

In [4]:
texts = [
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. ",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]

**참고(Reference)**

![](./images/top-ranked-embeddings.png)

- [(출처) Kor-IR: 한국어 검색을 위한 임베딩 벤치마크](https://github.com/teddylee777/Kor-IR?tab=readme-ov-file)

## HuggingFace Endpoint Embedding

`HuggingFaceEndpointEmbeddings` 는 내부적으로 InferenceClient를 사용하여 임베딩을 계산한다는 점에서 HuggingFaceEndpoint가 LLM에서 수행하는 것과 매우 유사합니다. 

In [12]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

model_name = "Qwen/Qwen3-Embedding-8B"

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
)

Document 임베딩은 `embed_documents()` 를 호출하여 생성할 수 있습니다.

In [13]:
%%time
# Document Embedding 수행
embedded_documents = hf_embeddings.embed_documents(texts)

CPU times: total: 15.6 ms
Wall time: 219 ms


HfHubHTTPError: 404 Client Error: Not Found for url: https://router.huggingface.co/hf-inference/pipeline/feature-extraction/Qwen/Qwen3-Embedding-8B (Request ID: Root=1-68a4f138-70e94064207468fd21907d92;3e45441b-da50-4f0d-94f7-3a0492b92865)

In [ ]:
print("[HuggingFace Endpoint Embedding]")
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

In [ ]:
# Document Embedding 수행
embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")

In [ ]:
len(embedded_query)

### 유사도 계산

**벡터 내적을 통한 유사도 계산**
- 벡터 내적(dot product)을 사용하여 유사도를 계산합니다. 

- 유사도 계산 공식:

$$ \text{similarities} = \mathbf{query} \cdot \mathbf{documents}^T $$

#### 벡터 내적의 수학적 의미

**벡터 내적 정의**

벡터 $\mathbf{a}$와 $\mathbf{b}$의 내적은 다음과 같이 정의됩니다:
$$ \mathbf{a} \cdot \mathbf{b} = \sum_{i=1}^{n} a_i b_i $$

**코사인 유사도와의 관계**

벡터 내적은 다음과 같은 성질을 가집니다.
$$ \mathbf{a} \cdot \mathbf{b} = \|\mathbf{a}\| \|\mathbf{b}\| \cos \theta $$

여기서,
- $\|\mathbf{a}\|$와 $\|\mathbf{b}\|$는 각각 벡터 $\mathbf{a}$와 $\mathbf{b}$의 크기(노름, Euclidean norm)입니다.
- $\theta$는 두 벡터 사이의 각도입니다.
- $\cos \theta$는 두 벡터 사이의 코사인 유사도입니다.

**벡터 내적의 유사도 해석**
내적 값이 클수록 (양의 큰 값일수록),
- 두 벡터의 크기($\|\mathbf{a}\|$와 $\|\mathbf{b}\|$)가 크고,
- 두 벡터 사이의 각도($\theta$)가 작으며 ($\cos \theta$가 1에 가까움),

이는 두 벡터가 유사한 방향을 가리키고, 크기가 클수록 더 유사하다는 것을 의미합니다.

**벡터의 크기(노름) 계산**

Euclidean norm 정의
벡터 $\mathbf{a} = [a_1, a_2, \ldots, a_n]$에 대해, Euclidean norm $\|\mathbf{a}\|$는 다음과 같이 정의됩니다:
$$ \|\mathbf{a}\| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2} $$

----

query 와 embedding_document 간의 유사도 계산

In [14]:
import numpy as np

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
np.array(embedded_query) @ np.array(embedded_documents).T

NameError: name 'embedded_query' is not defined

In [ ]:
sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]
sorted_idx

In [ ]:
print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()

## HuggingFace Embeddings



### `intfloat/multilingual-e5-large-instruct`

- [intfloat/multilingual-e5-large-instruct](https://huggingface.co/intfloat/multilingual-e5-large-instruct)
- [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large)

In [20]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"
# model_name = "intfloat/multilingual-e5-large"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cpu"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

In [21]:
%time
# Document
embedded_documents1 = hf_embeddings.embed_documents(texts)

CPU times: total: 0 ns
Wall time: 0 ns


In [22]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		intfloat/multilingual-e5-large-instruct


NameError: name 'embedded_documents' is not defined

## BGE-M3 임베딩

아래의 옵션 중 에러가 발생할 수 있는 옵션 설명

- `{"device": "mps"}`: GPU 대신 MPS를 사용하여 임베딩 계산을 수행합니다. (Mac 사용자)
- `{"device": "cuda"}`: GPU를 사용하여 임베딩 계산을 수행합니다. (Linux, Windows 사용자, 단 CUDA 설치 필요)
- `{"device": "cpu"}`: CPU를 사용하여 임베딩 계산을 수행합니다. (모든 사용자

In [23]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [24]:
%time
# Document
embedded_documents = hf_embeddings.embed_documents(texts)

CPU times: total: 0 ns
Wall time: 0 ns


In [25]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		BAAI/bge-m3
Dimension: 	1024


In [26]:
import numpy as np

embedded_query = hf_embeddings.embed_query("LangChain 에 대해서 알려주세요.")
embedded_documents = hf_embeddings.embed_documents(texts)

# 질문(embedded_query): LangChain 에 대해서 알려주세요.
np.array(embedded_query) @ np.array(embedded_documents).T

sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

print("[Query] LangChain 에 대해서 알려주세요.\n====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i}] {texts[idx]}")
    print()

[Query] LangChain 에 대해서 알려주세요.
[0] LangChain simplifies the process of building applications with large language models

[1] LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.

[2] 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 

[3] 안녕, 만나서 반가워.

[4] Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.



### `FlagEmbedding` 을 활용하는 방식

**참고**
- [FlagEmbedding - BGE-M3 Usage](https://github.com/FlagOpen/FlagEmbedding/tree/master/FlagEmbedding/BGE_M3#usage)


`FlagEmbedding` 에서 제공하는 세 가지 접근법을 조합하면, 더욱 강력한 검색 시스템을 구축할 수 있습니다.

- Dense Vector: BGE-M3의 다국어, 다중 작업 능력을 기반으로 함
- Lexical weight를 활용한 sparse embedding으로 정확한 단어 매칭을 수행
- ColBERT의 multi-vector 접근법으로 문맥을 고려한 세밀한 매칭 수행

In [27]:
# FlagEmbedding 설치
!pip install -qU FlagEmbedding

^C


  error: subprocess-exited-with-error
  
  × Building wheel for zlib-state (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      C:\Users\younghl\AppData\Local\Temp\pip-build-env-hhtwm2eo\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: OSI Approved :: MIT License
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ********************************************************************************
      
      !!
        self._finalize_license_expression()
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython

In [30]:
from FlagEmbedding import BGEM3FlagModel

model_name = "BAAI/bge-m3"
bge_embeddings = BGEM3FlagModel(
    model_name, use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.

bge_embedded = bge_embeddings.encode(
    texts,
    batch_size=12,
    max_length=8192,  # 이렇게 긴 길이가 필요하지 않은 경우 더 작은 값을 설정하여 인코딩 프로세스의 속도를 높일 수 있습니다.
)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [36]:
bge_embedded["dense_vecs"].shape

(5, 1024)

In [34]:
print(f"Model: \t\t{model_name}")
print(f"Dimension: \t{len(embedded_documents[0])}")

Model: 		BAAI/bge-m3
Dimension: 	1024


In [37]:
from FlagEmbedding import BGEM3FlagModel

bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.
bge_encoded = bge_flagmodel.encode(texts, return_dense=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [40]:
# 결과 출력(행, 열)
bge_encoded["dense_vecs"].shape

(5, 1024)

### Sparse Embedding (Lexical Weight)

Sparse embedding은 벡터의 대부분의 값이 0인 고차원 벡터를 사용하는 임베딩 방식입니다. Lexical weight를 활용한 방식은 단어의 중요도를 고려하여 임베딩을 생성합니다.

**작동 방식**
1. 각 단어에 대해 lexical weight(어휘적 가중치)를 계산합니다. 이는 TF-IDF나 BM25 같은 방법을 사용할 수 있습니다.
2. 문서나 쿼리의 각 단어에 대해, 해당 단어의 lexical weight를 사용하여 sparse vector의 해당 차원에 값을 할당합니다.
3. 결과적으로 문서나 쿼리는 대부분의 값이 0인 고차원 벡터로 표현됩니다.

**장점**
- 단어의 중요도를 직접적으로 반영할 수 있습니다.
- 특정 단어나 구문을 정확히 매칭할 수 있습니다.
- 계산이 상대적으로 빠릅니다.

In [41]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.
bge_encoded = bge_flagmodel.encode(texts, return_sparse=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [47]:
bge_encoded

{'dense_vecs': array([[-0.00492401,  0.02456352, -0.052077  , ...,  0.01638757,
         -0.02810185,  0.01218552],
        [-0.0191887 , -0.0181497 , -0.00382788, ...,  0.02405743,
          0.0128329 , -0.00620422],
        [ 0.0146354 , -0.02158708,  0.00848752, ..., -0.01035915,
          0.02252624,  0.01773892],
        [-0.04505184, -0.02406329, -0.01311002, ...,  0.02105702,
          0.02418931, -0.01861473],
        [-0.01293309, -0.00730584, -0.01213078, ..., -0.00564261,
          0.03795185,  0.00601621]], dtype=float32),
 'lexical_weights': [defaultdict(int,
              {'6888': 0.1594789,
               '195269': 0.28882447,
               '4': 0.1015741,
               '54807': 0.25699177,
               '2293': 0.14345205,
               '20451': 0.12553936,
               '713': 0.18402502,
               '11529': 0.17917302,
               '5': 0.11848412}),
  defaultdict(int,
              {'15786': 0.28657445,
               '62467': 0.29970926,
               '7

In [43]:
lexical_scores1 = bge_flagmodel.compute_lexical_matching_score(
    bge_encoded["lexical_weights"][0], bge_encoded["lexical_weights"][0]
)
lexical_scores2 = bge_flagmodel.compute_lexical_matching_score(
    bge_encoded["lexical_weights"][0], bge_encoded["lexical_weights"][1]
)
# 0 <-> 0
print(lexical_scores1)
# 0 <-> 1
print(lexical_scores2)

0.3015604503452778
0


### Multi-Vector (ColBERT)

ColBERT(Contextualized Late Interaction over BERT)는 문서 검색을 위한 효율적인 방법입니다. 이 방식은 문서와 쿼리를 여러 개의 벡터로 표현하는 multi-vector 접근법을 사용합니다.

**작동 방식**

1. 문서의 각 토큰에 대해 별도의 벡터를 생성합니다. 즉, 하나의 문서는 여러 개의 벡터로 표현됩니다.
2. 쿼리도 마찬가지로 각 토큰에 대해 별도의 벡터를 생성합니다.
3. 검색 시, 쿼리의 각 토큰 벡터와 문서의 모든 토큰 벡터 사이의 유사도를 계산합니다.
4. 이 유사도들을 종합하여 최종 검색 점수를 계산합니다.

**장점**
- 토큰 수준의 세밀한 매칭이 가능합니다.
- 문맥을 고려한 임베딩을 생성할 수 있습니다.
- 긴 문서에 대해서도 효과적으로 작동합니다.

In [48]:
bge_flagmodel = BGEM3FlagModel(
    "BAAI/bge-m3", use_fp16=True
)  # use_fp16을 True로 설정하면 약간의 성능 저하와 함께 계산 속도가 빨라집니다.
bge_encoded = bge_flagmodel.encode(texts, return_colbert_vecs=True)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [49]:
colbert_scores1 = bge_flagmodel.colbert_score(
    bge_encoded["colbert_vecs"][0], bge_encoded["colbert_vecs"][0]
)
colbert_scores2 = bge_flagmodel.colbert_score(
    bge_encoded["colbert_vecs"][0], bge_encoded["colbert_vecs"][1]
)
# 0 <-> 0
print(colbert_scores1)
# 0 <-> 1
print(colbert_scores2)

tensor(1.)
tensor(0.3748)
